# VirtuOffice Analytics - A `dash` app

By [Tom Keith](https://github.com/tomkeith) and [Angel Phanthanourak](https://github.com/angelphanth)

-----

In this notebook we will be creating an interactive dashboard app using the `plotly` and `dash` libraries. The dash app will visualize the fake weekly [Employee Satisfaction Survey](https://forms.gle/JGfmaBtNuPPmBPGu6) data generated in notebook *survey_data_generation.ipynb*. 

By employing `dash_core_components.Dropdown` and `dash_core_components.RangeSlider`, users will be able to update the `plotly.graph_objects`, `plotly.subplots` and `plotly.express` figures by selecting the company team and time periods of interest.

In [20]:
# Import the usual libraries 
import datetime as dt
import numpy as np
import pandas as pd

# Making the plots 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Importing dash libraries 
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

In [221]:
# Reading in the generated survey data
survey = pd.read_csv('survey_data.csv')

# Viewing the first 5 rows 
survey.head()

,employee_id,team,date,work_days,home_days,office_env,home_env,relationships,role,connections,involvement,lonliness,productivity
0,11,Production,2020-01-03,1,4,3,5,5,3,3,1,5,3
1,11,Production,2020-01-10,5,0,3,1,4,3,4,3,4,3
2,11,Production,2020-01-17,2,3,1,2,2,1,1,3,4,1
3,11,Production,2020-01-24,2,3,4,1,4,2,3,5,2,3
4,11,Production,2020-01-31,4,1,2,2,2,5,5,4,3,3


In [3]:
# Checking the column datatypes 
survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6240 entries, 0 to 6239
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   employee_id    6240 non-null   int64 
 1   team           6240 non-null   object
 2   date           6240 non-null   object
 3   work_days      6240 non-null   int64 
 4   home_days      6240 non-null   int64 
 5   office_env     6240 non-null   int64 
 6   home_env       6240 non-null   int64 
 7   relationships  6240 non-null   int64 
 8   role           6240 non-null   int64 
 9   connections    6240 non-null   int64 
 10  involvement    6240 non-null   int64 
 11  lonliness      6240 non-null   int64 
 12  productivity   6240 non-null   int64 
dtypes: int64(11), object(2)
memory usage: 633.9+ KB


In [222]:
# Changing 'date' column to date object
survey['date'] = [dt.datetime.strptime(x, '%Y-%m-%d').date() for x in survey['date']]

# Sanity check 
survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6240 entries, 0 to 6239
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   employee_id    6240 non-null   int64 
 1   team           6240 non-null   object
 2   date           6240 non-null   object
 3   work_days      6240 non-null   int64 
 4   home_days      6240 non-null   int64 
 5   office_env     6240 non-null   int64 
 6   home_env       6240 non-null   int64 
 7   relationships  6240 non-null   int64 
 8   role           6240 non-null   int64 
 9   connections    6240 non-null   int64 
 10  involvement    6240 non-null   int64 
 11  lonliness      6240 non-null   int64 
 12  productivity   6240 non-null   int64 
dtypes: int64(11), object(2)
memory usage: 633.9+ KB


In [223]:
# A list of the columns to change scale 
ratings = list(survey.columns[-8:])

# Iterate through each column
for question in ratings: 
    survey[question] = survey[question] / 5 * 100

# Sanity check 
survey.head()

,employee_id,team,date,work_days,home_days,office_env,home_env,relationships,role,connections,involvement,lonliness,productivity
0,11,Production,2020-01-03,1,4,60.0,100.0,100.0,60.0,60.0,20.0,100.0,60.0
1,11,Production,2020-01-10,5,0,60.0,20.0,80.0,60.0,80.0,60.0,80.0,60.0
2,11,Production,2020-01-17,2,3,20.0,40.0,40.0,20.0,20.0,60.0,80.0,20.0
3,11,Production,2020-01-24,2,3,80.0,20.0,80.0,40.0,60.0,100.0,40.0,60.0
4,11,Production,2020-01-31,4,1,40.0,40.0,40.0,100.0,100.0,80.0,60.0,60.0


In [224]:
# Changing employee_id to object type
survey['employee_id'] = survey['employee_id'].astype('str')

# Sanity check 
survey['employee_id'].unique()

array(['11', '12', '13', '14', '15', '16', '17', '18', '19', '110', '111',
       '112', '113', '114', '115', '116', '117', '118', '119', '120',
       '21', '22', '23', '24', '25', '26', '27', '28', '29', '210', '211',
       '212', '213', '214', '215', '216', '217', '218', '219', '220',
       '31', '32', '33', '34', '35', '36', '37', '38', '39', '310', '311',
       '312', '313', '314', '315', '316', '317', '318', '319', '320',
       '41', '42', '43', '44', '45', '46', '47', '48', '49', '410', '411',
       '412', '413', '414', '415', '416', '417', '418', '419', '420',
       '51', '52', '53', '54', '55', '56', '57', '58', '59', '510', '511',
       '512', '513', '514', '515', '516', '517', '518', '519', '520',
       '61', '62', '63', '64', '65', '66', '67', '68', '69', '610', '611',
       '612', '613', '614', '615', '616', '617', '618', '619', '620'],
      dtype=object)

In [225]:
# Average as mode for every week (all teams/entire company) 
modes_by_date = survey.groupby(['date']).agg(lambda x:x.value_counts().index[0]).reset_index()

# Sanity check 
modes_by_date

,date,employee_id,team,work_days,home_days,office_env,home_env,relationships,role,connections,involvement,lonliness,productivity
0,2020-01-03,52,Production,2,3,60.0,80.0,80.0,40.0,80.0,80.0,80.0,60.0
1,2020-01-10,52,Production,4,1,60.0,40.0,40.0,60.0,80.0,60.0,40.0,40.0
2,2020-01-17,52,Production,2,3,80.0,80.0,40.0,80.0,80.0,80.0,80.0,80.0
3,2020-01-24,52,Production,3,2,80.0,80.0,80.0,60.0,40.0,80.0,80.0,80.0
4,2020-01-31,52,Production,4,1,80.0,80.0,40.0,60.0,80.0,60.0,80.0,80.0
5,2020-02-07,52,Production,3,2,80.0,80.0,80.0,60.0,60.0,80.0,80.0,60.0
6,2020-02-14,52,Production,3,2,40.0,40.0,40.0,60.0,40.0,60.0,40.0,60.0
7,2020-02-21,52,Production,2,3,40.0,80.0,80.0,40.0,80.0,60.0,80.0,60.0
8,2020-02-28,52,Production,3,2,60.0,60.0,60.0,40.0,60.0,40.0,60.0,60.0
9,2020-03-06,52,Production,3,2,60.0,40.0,80.0,80.0,60.0,60.0,40.0,60.0


In [226]:
# Adding a column of hover_texts

# Empty list
hover_text = []

# Iterating through every row of the df 'modes_by_date'
for index, row in modes_by_date.iterrows():
    hover_text.append(('Week of {date}<br>'+
                       'Days WFH: {homedays}<br>'+
                       'Productivity: {product}%<br>'+
                       'Team Connection: {connect}%<br>'+
                       'Loneliness: {lonely}%<br>').format(date=row['date'], homedays=row['home_days'],
                                                           product=row['productivity'], connect=row['connections'],
                                                           lonely=row['lonliness']))

# Saving as new column 'text'
modes_by_date['text'] = hover_text

# Sanity check
modes_by_date.head()

,date,employee_id,team,work_days,home_days,office_env,home_env,relationships,role,connections,involvement,lonliness,productivity,text
0,2020-01-03,52,Production,2,3,60.0,80.0,80.0,40.0,80.0,80.0,80.0,60.0,Week of 2020-01-03<br>Days WFH: 3<br>Productiv...
1,2020-01-10,52,Production,4,1,60.0,40.0,40.0,60.0,80.0,60.0,40.0,40.0,Week of 2020-01-10<br>Days WFH: 1<br>Productiv...
2,2020-01-17,52,Production,2,3,80.0,80.0,40.0,80.0,80.0,80.0,80.0,80.0,Week of 2020-01-17<br>Days WFH: 3<br>Productiv...
3,2020-01-24,52,Production,3,2,80.0,80.0,80.0,60.0,40.0,80.0,80.0,80.0,Week of 2020-01-24<br>Days WFH: 2<br>Productiv...
4,2020-01-31,52,Production,4,1,80.0,80.0,40.0,60.0,80.0,60.0,80.0,80.0,Week of 2020-01-31<br>Days WFH: 1<br>Productiv...


In [227]:
# Creating a dictionary of dataframes for each team

# List of teams
team_names = survey['team'].unique()

# Creating the dataframes 
team_data = {team : survey.query("team == '%s'" %team).groupby(['date']).agg(lambda x:x.value_counts().index[0]).reset_index()
                 for team in team_names}

for team in team_names:
    
    # Empty list
    hover_text = []
    
    # Iterating through every row of the df 
    for index, row in team_data[team].iterrows():
        hover_text.append(('Week of {date}<br>'+
                           'Days WFH: {homedays}<br>'+
                           'Productivity: {product}%<br>'+
                           'Team Connection: {connect}%<br>'+
                           'Loneliness: {lonely}%<br>').format(date=row['date'], homedays=row['home_days'],
                                                               product=row['productivity'],
                                                               connect=row['connections'],
                                                               lonely=row['lonliness']))
    # Saving as new column 'text'
    team_data[team]['text'] = hover_text

# Sanity check: looking at the production team - modes for every week 
team_data['Production']

,date,employee_id,team,work_days,home_days,office_env,home_env,relationships,role,connections,involvement,lonliness,productivity,text
0,2020-01-03,110,Production,1,4,60.0,80.0,40.0,60.0,80.0,60.0,40.0,60.0,Week of 2020-01-03<br>Days WFH: 4<br>Productiv...
1,2020-01-10,110,Production,2,3,60.0,40.0,80.0,60.0,60.0,60.0,40.0,40.0,Week of 2020-01-10<br>Days WFH: 3<br>Productiv...
2,2020-01-17,110,Production,4,1,60.0,60.0,40.0,60.0,80.0,80.0,80.0,60.0,Week of 2020-01-17<br>Days WFH: 1<br>Productiv...
3,2020-01-24,110,Production,3,3,60.0,20.0,80.0,60.0,60.0,80.0,60.0,60.0,Week of 2020-01-24<br>Days WFH: 3<br>Productiv...
4,2020-01-31,110,Production,4,3,60.0,60.0,80.0,60.0,80.0,80.0,80.0,80.0,Week of 2020-01-31<br>Days WFH: 3<br>Productiv...
5,2020-02-07,110,Production,3,2,100.0,100.0,80.0,60.0,60.0,80.0,80.0,60.0,Week of 2020-02-07<br>Days WFH: 2<br>Productiv...
6,2020-02-14,110,Production,3,3,80.0,60.0,40.0,60.0,40.0,80.0,40.0,60.0,Week of 2020-02-14<br>Days WFH: 3<br>Productiv...
7,2020-02-21,110,Production,5,4,80.0,80.0,80.0,20.0,80.0,40.0,80.0,60.0,Week of 2020-02-21<br>Days WFH: 4<br>Productiv...
8,2020-02-28,110,Production,5,4,80.0,60.0,60.0,80.0,60.0,60.0,60.0,100.0,Week of 2020-02-28<br>Days WFH: 4<br>Productiv...
9,2020-03-06,110,Production,4,1,60.0,40.0,20.0,60.0,60.0,80.0,40.0,80.0,Week of 2020-03-06<br>Days WFH: 1<br>Productiv...


In [255]:
# Getting a list of dropdown options 
features = list(team_names)

# Adding option for entire company
features.append('All')

# Sanity check
features

['Production', 'Sales', 'Marketing', 'RD', 'HR', 'Accounting', 'All']

In [256]:
# Dropdown options
opts = [{'label': i, 'value': i} for i in features]

# Sanity check 
opts

[{'label': 'Production', 'value': 'Production'},
 {'label': 'Sales', 'value': 'Sales'},
 {'label': 'Marketing', 'value': 'Marketing'},
 {'label': 'RD', 'value': 'RD'},
 {'label': 'HR', 'value': 'HR'},
 {'label': 'Accounting', 'value': 'Accounting'},
 {'label': 'All', 'value': 'All'}]

In [295]:
# Range slider options saved to 'dates'
dates = list(modes_by_date['date'][::3])

# Sanity check 
dates[0:5]

[datetime.date(2020, 1, 3),
 datetime.date(2020, 1, 24),
 datetime.date(2020, 2, 14),
 datetime.date(2020, 3, 6),
 datetime.date(2020, 3, 27)]

In [298]:
# Initialize the app
app = dash.Dash(external_stylesheets=['https://stackpath.bootstrapcdn.com/bootswatch/4.4.1/materia/bootstrap.min.css'])

# Creating a subplot
fig = make_subplots(2, 1, shared_xaxes=True, vertical_spacing=0.08)

# To scale marker sizes by days WFH
sizeref = 2.*modes_by_date['home_days'].max()/(10**2)

# The traces
trace_1 = go.Scatter(x=modes_by_date['date'], y=modes_by_date['productivity'], mode='lines+markers', 
                     text=modes_by_date['text'], line=dict(width=3,color='#2d5986'), 
                     marker=dict(size=10, line=dict(color='#ffffff', width=1)),
                     name='<b>Company</b> Productivity')

trace_2 = go.Scatter(x=modes_by_date['date'], y=modes_by_date['lonliness'], mode='lines+markers', 
                     text=[str(x)+' WFH days' for x in modes_by_date['home_days']],
                     marker=dict(size=modes_by_date['home_days'],
                                 sizeref=sizeref, color='#00b3b3'), name='Loneliness')

trace_3 = go.Bar(x=modes_by_date['date'], y=modes_by_date['connections'], opacity=0.8, 
                 name='Team Connection', marker=dict(color='#9fbfdf', line=dict(color='#ffffff', width=1)))

# Adding traces to subplot
fig.append_trace(trace_1, 2, 1)
fig.append_trace(trace_3, 2, 1)
fig.append_trace(trace_2, 1, 1)

# Update subplot layout
fig.update_layout(legend_orientation='h', legend=dict(x=0, y=1.1), hovermode='closest', autosize=True, 
                  yaxis=dict(range=[0, 100], side="right", type="linear", zeroline=False),
                  yaxis2=dict(range=[0, 100], side="right", type="linear", zeroline=False), 
                  width=1200, height=600, font=dict(size=16))

# Creating sunburst plots for employee satisfaction

# List of most frequent scores 
sun_vals = [modes_by_date['office_env'].value_counts().index[0], 
            modes_by_date['home_env'].value_counts().index[0], 
            modes_by_date['relationships'].value_counts().index[0], 
            modes_by_date['role'].value_counts().index[0]]

# List of colours
sun_color = ['#000000', '#99ccff', '#ace600', '#ffe6ff', '#cc99ff']

# Creating dictionaries of data for sunbursts
sun_data = [dict(character = [sun_vals[i], ' '], parent = ['', sun_vals[i]], value = [100, sun_vals[i]]) 
                              for i in range(0,4)]

# Creating the sunburst plots 
sun0, sun1, sun2, sun3 = [px.sunburst(sun_data[i], names='character', parents='parent', values='value', 
                                      branchvalues='total', color='character', 
                                      color_discrete_map={sun_data[i]['character'][0]:sun_color[0], 
                                                          sun_data[i]['character'][1]:sun_color[i+1]}) 
                          for i in range(0,4)]

# Saving plots to a list 
sun_charts = [sun0, sun1, sun2, sun3]

# Updating the layouts of the plots 
for i in range(0,4):
    sun_charts[i].update_traces(textfont_size=50, textfont_color=sun_color[i+1])
    sun_charts[i].update_layout(height=200, margin=dict(l=20, r=20, t=20, b=20))

    
# Create the dash layout
app.layout = html.Div([
                # Header
                html.Div([
                    html.H1("VirtuOffice Analytics", style={'fontSize':'50px','color':'#ffffff'}),
                    html.P("BrainStation x Microsoft Hackathon", style={'color':'#ffffff'})
                         ], style = {'padding' : '50px', 'backgroundColor' : '#264d73'}),
                # Satisfaction charts title
                html.Div([
                    html.H2("Employee Satisfaction with the following:")], style = {'padding' : '20px'}),
                # Sunbursts
                dbc.Row(
                    [dbc.Col(html.Div([
                        html.H5('Office Culture', style={'textAlign':'center'}),
                        dcc.Graph(id='g1', figure=sun0, config={'displayModeBar': False})
                    ]), width={'size':3}),

                     dbc.Col(html.Div([
                        html.H5('WFH Environment', style={'textAlign':'center'}),
                        dcc.Graph(id='g2', figure=sun1, config={'displayModeBar': False})
                    ]), width={'size':3}),

                     dbc.Col(html.Div([
                        html.H5('Working Relationships', style={'textAlign':'center'}),
                        dcc.Graph(id='g3', figure=sun2, config={'displayModeBar': False})
                    ]), width={'size':3}),

                    dbc.Col(html.Div([
                        html.H5('Work Roles', style={'textAlign':'center'}),
                        dcc.Graph(id='g4', figure=sun3, config={'displayModeBar': False})
                    ]), width={'size':3})]),
                # xy subplot
                dbc.Row(
                    [dbc.Col(html.Div([
                        html.H2('Lonliness and Days WFH, Productivity and Team Connection over Time'),
                        html.P("Interact with all charts by selecting a team and/or a date range of interest below."),
                        dcc.Graph(id='plot',figure=fig, config={'displayModeBar': False}), 
            
                        # Dropdown menu
                        html.P([
                            html.Label("Choose a Team", style={'fontSize':'20px'}),
                            dcc.Dropdown(id = 'opt', options = opts, value = features[-1])
                                ], style = {'width': '400px', 'fontSize' : '14px',
                                            'padding-left' : '100px', 'display': 'inline-block'}),

                        # Range slider
                        html.P([
                            html.Label("Time Period", style={'fontSize':'20px'}),
                            dcc.RangeSlider(id = 'slider',
                                            marks = {i : {'label':dates[i], 
                                                          'style':{'fontSize':'9px', 
                                                                   'color':'#b3b3b3',
                                                                   'transform':'rotate(45deg)'}} 
                                                     for i in range(0, 18)},
                                            min = 0,
                                            max = 17,
                                            value = [1, 16])
                                ], style = {'width' : '80%',
                                            'padding-left' : '100px',
                                            'display': 'inline-block'})], 
                        style = {'padding-left':'50px'}), width={'size':12})])])        


# Multi-Output Callback functions
@app.callback([Output('plot', 'figure'), Output('g1', 'figure'), 
               Output('g2', 'figure'), Output('g3', 'figure'), Output('g4', 'figure')],
              [Input('opt', 'value'), Input('slider', 'value')])
   
def update_figure(input1, input2):
    
    # When a team is selected from the dropdown 
    if input1 != 'All':
        
        # Filter the selected team's df
        st2 = team_data[input1][(team_data[input1]['date'] > dates[input2[0]]) &\
                                (team_data[input1]['date'] < dates[input2[1]])]
        
        # Filter the entire company's df
        st1 = modes_by_date[(modes_by_date['date'] > dates[input2[0]]) & (modes_by_date['date'] < dates[input2[1]])]
        
        # Updating the traces of the subplot 
        trace_1 = go.Scatter(x=st2['date'], y=st2['productivity'], mode='lines+markers', 
                             text=st2['text'], line=dict(width=3,color='#2d5986'), 
                             marker=dict(size=10, line=dict(color='#ffffff', width=1)),
                             name='<b>Team</b> Productivity')

        trace_2 = go.Scatter(x=st2['date'], y=st2['lonliness'], mode='lines+markers', 
                             text=[str(x)+' WFH days' for x in st2['home_days']],
                             marker=dict(size=modes_by_date['home_days'],
                                         sizeref=sizeref, color='#00b3b3'), name='Loneliness')

        trace_3 = go.Bar(x=st2['date'], y=st2['connections'], opacity=0.8, 
                         name='Team Connection', marker=dict(color='#9fbfdf', line=dict(color='#ffffff', width=1)))
        
        # Compile the subplots
        fig = make_subplots(2, 1, shared_xaxes=True, vertical_spacing=0.08)
        fig.append_trace(trace_1, 2, 1)
        fig.append_trace(trace_3, 2, 1)
        fig.append_trace(trace_2, 1, 1)
        fig.update_layout(legend_orientation='h', legend=dict(x=0, y=1.1), hovermode='closest', autosize=True, 
                          yaxis=dict(range=[0, 100], side="right", type="linear", zeroline=False),
                          yaxis2=dict(range=[0, 100], side="right", type="linear", zeroline=False), 
                          width=1200, height=600, font=dict(size=16))
    
        # Updating the sunbursts 
        sun_vals2 = [st2['office_env'].value_counts().index[0], 
                     st2['home_env'].value_counts().index[0], 
                     st2['relationships'].value_counts().index[0], 
                     st2['role'].value_counts().index[0]]
        
        # Creating dictionaries of data for sunbursts
        sun_data2 = [dict(character = [sun_vals2[i], ' '], parent = ['', sun_vals2[i]], value = [100, sun_vals2[i]]) 
                     for i in range(0,4)]
        # Updated plots
        sun0, sun1, sun2, sun3 = [px.sunburst(sun_data2[i], names='character', parents='parent', values='value', 
                                              branchvalues='total', color='character', 
                                              color_discrete_map={sun_data2[i]['character'][0]:sun_color[0], 
                                                                  sun_data2[i]['character'][1]:sun_color[i+1]}) 
                                  for i in range(0,4)]
        
        # Saving plots to a list 
        sun_charts2 = [sun0, sun1, sun2, sun3]

        # Updating the layouts of the plots 
        for i in range(0,4):
            sun_charts2[i].update_traces(textfont_size=50, textfont_color=sun_color[i+1])
            sun_charts2[i].update_layout(height=200, margin=dict(l=20, r=20, t=20, b=20))
    
    # If 'All' selected from dropdown
    else:
        
        # Filter the selected team's df
        st2 = modes_by_date[(modes_by_date['date'] > dates[input2[0]]) & (modes_by_date['date'] < dates[input2[1]])]
        
        # Updating dates of original traces
        trace_1 = go.Scatter(x=st2['date'], y=st2['productivity'], mode='lines+markers', 
                             text=st2['text'], line=dict(width=3,color='#2d5986'), 
                             marker=dict(size=10, line=dict(color='#ffffff', width=1)),
                             name='<b>Company</b> Productivity')

        trace_2 = go.Scatter(x=st2['date'], y=st2['lonliness'], mode='lines+markers', 
                             text=[str(x)+' WFH days' for x in st2['home_days']],
                             marker=dict(size=modes_by_date['home_days'],
                                         sizeref=sizeref, color='#00b3b3'), name='Loneliness')

        trace_3 = go.Bar(x=st2['date'], y=st2['connections'], opacity=0.8, 
                         name='Team Connection', marker=dict(color='#9fbfdf', line=dict(color='#ffffff', width=1)))

        # Compiling the new subplots
        fig = make_subplots(2, 1, shared_xaxes=True, vertical_spacing=0.08)
        fig.append_trace(trace_1, 2, 1)
        fig.append_trace(trace_3, 2, 1)
        fig.append_trace(trace_2, 1, 1)
        fig.update_layout(legend_orientation='h', legend=dict(x=0, y=1.1), hovermode='closest', autosize=True, 
                          yaxis=dict(range=[0, 100], side="right", type="linear", zeroline=False),
                          yaxis2=dict(range=[0, 100], side="right", type="linear", zeroline=False), 
                          width=1200, height=600, font=dict(size=16))

        # Updating sunbursts 
        sun_vals2 = [st2['office_env'].value_counts().index[0], 
                     st2['home_env'].value_counts().index[0], 
                     st2['relationships'].value_counts().index[0], 
                     st2['role'].value_counts().index[0]]
        
        # Creating dictionaries of data for sunbursts
        sun_data2 = [dict(character = [sun_vals2[i], ' '], parent = ['', sun_vals2[i]], value = [100, sun_vals2[i]]) 
                     for i in range(0,4)]
        
        # Update plots
        sun0, sun1, sun2, sun3 = [px.sunburst(sun_data2[i], names='character', parents='parent', values='value', 
                                              branchvalues='total', color='character', 
                                              color_discrete_map={sun_data2[i]['character'][0]:sun_color[0], 
                                                                  sun_data2[i]['character'][1]:sun_color[i+1]}) 
                                  for i in range(0,4)]
        
        # Saving plots to a list 
        sun_charts2 = [sun0, sun1, sun2, sun3]

        # Updating the layouts of the plots 
        for i in range(0,4):
            sun_charts2[i].update_traces(textfont_size=50, textfont_color=sun_color[i+1])
            sun_charts2[i].update_layout(height=200, margin=dict(l=20, r=20, t=20, b=20))

    return fig, sun0, sun1, sun2, sun3

  
# Server clause
if __name__ == '__main__':
    app.run_server(debug = True, use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on htt